In [153]:
def bitlist_to_hexstring(bitlist):
    return str(hex(int("".join(map(lambda x: str(x), bitlist)), base=2))[2:]).zfill(len(bitlist)//4)

def hexstring_to_bitlist(hexstring, resulting_length):
    if hexstring[-1] == 'L': 
        hexstring = hexstring[:-1]
    bits = bin(int(hexstring, base=16))
    bitlist = map(lambda x: int(x), list(bits[2:]))
    bitlist = [0] * (resulting_length-len(bitlist))+ bitlist
    return bitlist

hexstring_to_bitlist("00", 5)


[0, 0, 0, 0, 0]

In [58]:
sbox = []
for x in range(1, 9):
    input_table = [list() for i in range(4)]
    input_file = open("../../sbox%s.txt" % x, "r")
    for i in range(4):
        for line_index in range(16):
            input_table[i].append(int(input_file.readline()))
    sbox.append(input_table)
sbox

[[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
  [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
  [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
  [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
 [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
  [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
  [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
  [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
 [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
  [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
  [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
  [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
 [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
  [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
  [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
  [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
 [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
  [14, 11,

In [71]:
expansion_table = [32,  1,  2,  3,  4,  5,
                    4,  5,  6,  7,  8,  9,
                    8,  9, 10, 11, 12, 13,
                   12, 13, 14, 15, 16, 17,
                   16, 17, 18, 19, 20, 21,
                   20, 21, 22, 23, 24, 25,
                   24, 25, 26, 27, 28, 29,
                   28, 29, 30, 31, 32,  1]

# input is a bit list
def expansion(input):
    result = []
    for x in expansion_table:
        result.append(input[x-1])
    return result

print len(expansion([0] * 32))

48


In [184]:
def rotate(l, factor):
    return l[factor:] + l[:factor]

rotate([1,2,3,4], 1)

[2, 3, 4, 1]

In [143]:
pc1_table = [57, 49, 41, 33, 25, 17,  9,  1,
             58, 50, 42, 34, 26, 18, 10,  2,
             59, 51, 43, 35, 27, 19, 11,  3,
             60, 52, 44, 36, 63, 55, 47, 39,
             31, 23, 15,  7, 62, 54, 46, 38,
             30, 22, 14,  6, 61, 53, 45, 37,
             29, 21, 13,  5, 28, 20, 12,  4]

def remove_parity_bits(raw_key):
    resulting_key = []
    for x in pc1_table:
        resulting_key.append(raw_key[x-1])
    return resulting_key

In [192]:
pc2_table = [14, 17, 11, 24,  1,  5,  3, 28,
             15,  6, 21, 10, 23, 19, 12,  4,
             26,  8, 16,  7, 27, 20, 13,  2,
             41, 52, 31, 37, 47, 55, 30, 40,
             51, 45, 33, 48, 44, 49, 39, 56,
             34, 53, 46, 42, 50, 36, 29, 32]

#input_key is a bit list
def gen_key_for_round(input_key, round):
    c_part = input_key[0:28]
    d_part = input_key[28:56]
    for i in range(round):
        if i == 0 or i == 1 or i == 8 or i == 15:
            c_part = rotate(c_part, 1)
            d_part = rotate(d_part, 1)
        else:
            c_part = rotate(c_part, 2)
            d_part = rotate(d_part, 2)
    
    rotated_key = c_part + d_part
    resulting_key = []
    for x in pc2_table:
        resulting_key.append(rotated_key[x-1])
    return resulting_key
    
print gen_key_for_round([0] * 56, 3)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [62]:
def xor_two_bit_lists(bit_list_a, bit_list_b):
    return map(lambda x: x[0] ^ x[1], zip(bit_list_a, bit_list_b))

In [63]:
xor_two_bit_lists([0,0,1,1], [0,1,0,1])

[0, 1, 1, 0]

In [64]:
def bitlist_to_int(bit_list):
    value = 0
    for bit in bit_list:
        value = (value << 1) | bit
    return value

def int_to_bitlist(integer):
    bitlist = []
    for x in range(4):
        bitlist = [integer & 0x1] + bitlist
        integer = integer >> 1
    return bitlist

int_to_bitlist(13)

[1, 1, 0, 1]

In [65]:
int_to_bitlist(sbox[0][0][0])

[1, 1, 1, 0]

In [73]:
def use_sboxes_on_bit_list(bit_list):
    result = []
    for x in range(8):
        value = bitlist_to_int(bit_list[x*6:(x+1)*6])
        middle = (value >> 1) & 15
        ends = ((value >> 5) << 1) + (value & 1)
        result = result + int_to_bitlist(sbox[x][ends][middle])
    return result
        
print use_sboxes_on_bit_list([0, 1] * 24)

[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]


In [67]:
permute_table_p = [16,  7, 20, 21, 29, 12, 28, 17,
                    1, 15, 23, 26,  5, 18, 31, 10,
                    2,  8, 24, 14, 32, 27,  3,  9,
                   19, 13, 30,  6, 22, 11,  4, 25]

def permute_p(bitlist):
    result_list = []
    for x in permute_table_p:
        result_list.append(bitlist[x-1])
    return result_list

In [78]:
def des_f_function(k, e, round):
    expanded_e = expansion(e)
    generated_key = gen_key_for_round(k, round)
    xored_list = xor_two_bit_lists(expanded_e, generated_key)
    sboxed_list = use_sboxes_on_bit_list(xored_list)
    permuted_list = permute_p(sboxed_list)
    return permuted_list

print bitlist_to_hexstring(des_f_function([1] * 56, [1] * 32, 0))
print bitlist_to_hexstring(des_f_function([0] * 56, hexstring_to_bitlist("d8d8dbbc"), 1))

d8d8dbbc
e73aed4f


In [156]:
initial_permutation_table = [58, 50, 42, 34, 26, 18, 10,  2,
                             60, 52, 44, 36, 28, 20, 12,  4,
                             62, 54, 46, 38, 30, 22, 14,  6,
                             64, 56, 48, 40, 32, 24, 16,  8,
                             57, 49, 41, 33, 25, 17,  9,  1,
                             59, 51, 43, 35, 27, 19, 11,  3,
                             61, 53, 45, 37, 29, 21, 13,  5,
                             63, 55, 47, 39, 31, 23, 15,  7]

def initial_permutation(bitlist):
    result = []
    for x in initial_permutation_table:
        result.append(bitlist[x-1])
    return result

In [69]:
final_permutation_table = [40, 8, 48, 16, 56, 24, 64, 32,
                           39, 7, 47, 15, 55, 23, 63, 31,
                           38, 6, 46, 14, 54, 22, 62, 30,
                           37, 5, 45, 13, 53, 21, 61, 29,
                           36, 4, 44, 12, 52, 20, 60, 28,
                           35, 3, 43, 11, 51, 19, 59, 27,
                           34, 2, 42, 10, 50, 18, 58, 26,
                           33, 1, 41,  9, 49, 17, 57, 25]

def final_permutation(bitlist):
    result = []
    for x in final_permutation_table:
        result.append(bitlist[x-1])
    return result

In [190]:
# plaintext is 64 bit long
def des_encrypt(k, plaintext):
    #initial permutation
    l = plaintext[0:32]
    r = plaintext[32:64]
    for x in range(16):
        temp = r
        print bitlist_to_hexstring(des_f_function(k, r, x+1))
        r = xor_two_bit_lists(l, des_f_function(k, r, x+1))
        l = temp
    return final_permutation(r + l)

print bitlist_to_hexstring(des_encrypt([0] * 56, [0] * 64))

d8d8dbbc
e73aed4f
8322b01a
b97f0f18
d0e31046
a3ac6b1d
92c670bc
6d939aba
91c6070c
b5ab428c
7bf69d2e
b332c66e
a9b23234
936f3a18
e1b7ae88
19ac2af6
8ca64de9c1b123a7L


In [204]:
from random import randint
input_value = hex(randint(0, 2**64))
r = initial_permutation(hexstring_to_bitlist(input_value, 64))[32:64]
input_key = hex(randint(0, 2**64))
real_key = remove_parity_bits(hexstring_to_bitlist(input_key, 64))
input_round = randint(1, 16)

print "plaintext e         : %s" % input_value
print "key k               : %s" % input_key
print "key k without parity: 0x%s" % bitlist_to_hexstring(real_key)
print "round               : %d" % input_round
print "r                   : 0x%s" % bitlist_to_hexstring(r)
print "round_key           : 0x%s" % bitlist_to_hexstring(gen_key_for_round(real_key, 1))
print "f_function result   : 0x%s" % bitlist_to_hexstring(des_f_function(real_key, r, input_round))

plaintext e         : 0xffef6432ec8804d2L
key k               : 0xb6c31861602ac950L
key k without parity: 0x43da3982301645
round               : 3
r                   : 0xb31f338b
round_key           : 0xb84e540644e0
f_function result   : 0x6223b2db


In [195]:
from random import randint
input_value = hex(randint(0, 2**64))
lr = initial_permutation(hexstring_to_bitlist(input_value, 64))
input_key = hex(randint(0, 2**64))
real_key = remove_parity_bits(hexstring_to_bitlist(input_key, 64))
input_round = 1# randint(1, 16)

print "plaintext e         : %s" % input_value
print "key k               : %s" % input_key
print "key k without parity: %s" % bitlist_to_hexstring(real_key)
print "round               : %d" % input_round
print "r                   : %s" % bitlist_to_hexstring(r)
print "round_key           : %s" % bitlist_to_hexstring(gen_key_for_round(real_key, 1))
print "des_encrypt result  : 0x%s" % bitlist_to_hexstring(des_encrypt(real_key, lr))

plaintext e         : 0xe336a9e70264e761L
key k               : 0x230ab6d4eb05d23d
key k without parity: 5c5895c57ac92c
round               : 1
r                   : 1849f49d
round_key           : 3e2f20e928ce
0cb33929
097c3b82
8024fe75
39a7f650
4a964d2b
9883ca8f
3aacb07e
a08628b1
283ffd1b
dbe4f981
12113666
2caa6e25
321a2784
3d5d9988
aa4bdde7
01e2beb7
des_encrypt result  : 0x8cc6c147a6893af6L
